In [4]:
import os
os.system("pip install lingam")
os.system("pip install fsspec")
os.system("pip install s3fs")
os.system("pip install pyarrow")

import random as rnd
from scipy.spatial import distance
import pandas as pd
from sklearn import preprocessing
import warnings
import sys
import argparse
import ast
import time
import json

import boto3

#German
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.model_selection import train_test_split

import pickle

import numpy as np
import pandas as pd
import graphviz
import lingam
from lingam.utils import print_causal_directions, print_dagc, make_dot

import random as rnd

# from IPython.display import display

import warnings

warnings.filterwarnings('ignore')

s3 = boto3.client('s3')

#Used for ordering evaluations
class individual:
    def __init__(self, index, score, distance, num_changes, aval_norm, dist_norm, predict_proba):
        self.index = index #Indicates the instance's position in the dataframe
        self.score = score #Indicates the score in relation to the proximity of the class boundary
        self.distance = distance #Indicates the distance from the original instance
        self.num_changes = num_changes #Indicates the number of changes for class change
        self.aval_norm = aval_norm #Indicates the final fitness with standardized metrics
        self.dist_norm = dist_norm #Indicates the normalized distance (distance and number of changes)
        self.predict_proba = predict_proba #Indicates de individual's class
    def __repr__(self):
        return repr((self.index, self.score, self.distance, self.num_changes, self.aval_norm, self.dist_norm, self.predict_proba))

class counter_change:
    def __init__(self, column, value):
        self.column = column 
        self.value = value
    def __eq__(self, other):
        if self.column == other.column and self.value == other.value:
            return True
        else:
            return False
    def __repr__(self):
        return repr((self.column, self.value))    

#Used to generate a random value in the mutation operation
class feature_range:
    def __init__(self, column, col_type, min_value, max_value):
        self.column = column 
        self.col_type = col_type
        self.min_value = min_value
        self.max_value = max_value

    #Returns a random value to perform mutation operation
    def get_random_value(self):
        if self.col_type == 'int64' or self.col_type == 'int' or self.col_type == 'int16' or self.col_type == 'int8' or (self.col_type == 'uint8'):
            value = rnd.randint(self.min_value, self.max_value)
        else:  
            value = round(rnd.uniform(self.min_value, self.max_value), 2)
        return value
    
    #Checks if the attribute has only one value.
    def unique_value(self):
        if self.min_value != self.max_value:
            return False
        else:  
            return True    

    def __repr__(self):
        return repr((self.column, self.col_type, self.min_value, self.max_value)) 
        
class CSSE(object):
    
    def __init__(self, input_dataset, model, static_list = [], K = 3, num_gen = 30, pop_size = 100, per_elit = 0.1, cros_proba = 0.8, mutation_proba = 0.1, L1 = 1, L2 = 1):
        #User Options
        self.static_list = static_list #List of static features
        self.K = K #Number of counterfactuals desired
        #Model
        self.input_dataset = input_dataset
        self.model = model
        #GA Parameters
        self.num_gen = num_gen
        self.pop_size = pop_size
        self.per_elit = per_elit
        self.cros_proba = cros_proba
        self.mutation_proba = mutation_proba
        #Objective function parameters
        self.L1 = L1 #weight assigned the distance to the original instance
        self.L2 = L2 #weight assigned the number of changes needed in the original instance   
    
    #Get which index in the SHAP corresponding to the current class
    def getBadClass(self):   
        if self.current_class == self.model.classes_[0]:
            ind_cur_class = 0
        else:
            ind_cur_class = 1
        
        return ind_cur_class
    
    #Gets the valid values range for each feature
    def getFeaturesRange(self):
        features_range = []
       
        for i in range (0, self.input_dataset.columns.size):
            col_name = self.input_dataset.columns[i]
            col_type = self.input_dataset[col_name].dtype
            min_value = min(self.input_dataset[col_name])
            max_value = max(self.input_dataset[col_name])
            
            feature_range_ind = feature_range(col_name, col_type, min_value, max_value)
            features_range.append(feature_range_ind)
        
        return features_range
       
    def getMutationValue(self, currentValue, index, ind_feature_range):
        new_value = ind_feature_range.get_random_value()
        
        while currentValue == new_value:
            new_value = ind_feature_range.get_random_value()
        
        return new_value
    
    def equal(self, individual, population):
        aux = 0
        for i in range ( 1, len(population)):
            c = population.loc[i].copy()
            dst = distance.euclidean(individual, c)
            if dst == 0:
                aux = 1
        
        return aux

    def getPopInicial (self, df, features_range): 
        #The reference individual will always be in the 0 position of the df - so that it is normalized as well (it will be used later in the distance function)
        df.loc[0] = self.original_ind.copy()
        
        #Counting numbers of repeated individuals
        number_repetitions = 0
        
        #One more position is used because the zero position was reserved for the reference individual
        while len(df) < self.pop_size + 1:
            #Draw a feature to change
            index_a = rnd.randint( 0, self.input_dataset.columns.size - 1 )
            while df.columns[index_a] in self.static_list:
                index_a = rnd.randint( 0, self.input_dataset.columns.size - 1 )
                
            if not features_range[index_a].unique_value():
                #Mutation
                mutant = self.original_ind.copy()

                new_value =  self.getMutationValue(mutant.iloc[index_a], index_a, features_range[index_a])
                mutant.iloc[index_a] = new_value

                ni = self.equal(mutant, df)
                if ni == 0:
                    df.loc[len(df)] = mutant.copy()
                else:
                    #Assesses whether the GA is producing too many repeated individuals.
                    number_repetitions = number_repetitions + 1
                    if number_repetitions == 2*self.pop_size:
                        self.pop_size = round(self.pop_size - self.pop_size*0.1)
                        self.mutation_proba = self.mutation_proba + 0.1
                        #print('Adjusting population size...', self.pop_size)
                        number_repetitions = 0
    
    #Complete the standardized proximity and similarity assessments for each individual
    def getNormalEvaluation(self, evaluation, aval_norma):
        scaler2 = preprocessing.MinMaxScaler()
        aval_norma2 = scaler2.fit_transform(aval_norma)
    
        i = 0
        while i < len(evaluation):
            evaluation[i].aval_norm = self.L1*aval_norma2[i,0] + self.L2*aval_norma2[i,1] + aval_norma2[i,2]
            evaluation[i].dist_norm = self.L1*aval_norma2[i,0] + self.L2*aval_norma2[i,1]
        
            i = i + 1
    
    def numChanges(self, ind_con):
        num = 0
        for i in range(len(self.original_ind)):
            if self.original_ind[i] != ind_con[i]:
                num = num + 1
        
        return num
        
    def fitness(self, population, evaluation, ind_cur_class):
        def getProximityEvaluation (proba):
            #Penalizes the individual who is in the negative class
            if proba < 0.5:
                predict_score = 0
            else:
                predict_score= proba
             
            return predict_score
               
        #Calculates similarity to the original instance
        def getEvaluationDist (ind, X_train_minmax):
            #Normalizes the data so that the different scales do not bias the distance
            a = X_train_minmax[0]
            b = X_train_minmax[ind]
            dst = distance.euclidean(a, b)
  
            return dst
        
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
        
            predict_proba = self.model.predict_proba(population)
                    
        #Calculating the distance between instances
        scaler = preprocessing.MinMaxScaler()
        X_train_minmax = scaler.fit_transform(population)
    
        i = 0
        aval_norma = [] 
        while i < len(population):
            proximityEvaluation = getProximityEvaluation(predict_proba[i, ind_cur_class])
            evaldist = getEvaluationDist(i, X_train_minmax)
            #The original individual is in the 1st position
            numChanges = self.numChanges(population.loc[i])
        
            ind = individual(i, proximityEvaluation, evaldist, numChanges, 0, 0, predict_proba[i, ind_cur_class])
            aval_norma.append([evaldist, numChanges, proximityEvaluation])
            evaluation.append(ind)
            i = i + 1

        self.getNormalEvaluation(evaluation, aval_norma)
       
    #Given a counterfactual solution returns the list of modified columns
    def getColumns(self, counter_solution):
        colums = []
        for j in range (0, len(counter_solution)):
            colums.append(counter_solution[j].column)
        
        return colums      
             
    #Checks if the new solution is contained in the solutions already found
    def contained_solution(self, original_instance, current_list, current_column_list, new_solution, new_column_solution):
        contained = False
        for i in range (0, len(current_list)):              
            if set(current_column_list[i]).issubset(new_column_solution):
                for j in range (0, len(current_list[i])):
                    pos = new_column_solution.index(current_list[i][j].column)
                    distancia_a = abs(original_instance[current_list[i][j].column] - current_list[i][j].value)
                    distancia_b = abs(original_instance[current_list[i][j].column] - new_solution[pos].value)
                    if distancia_b >= distancia_a:
                        contained = True

        return contained
      
    def elitism(self, evaluation, df, parents):
         
        num_elit = round(self.per_elit*self.pop_size)
        
        aval = []
        aval = evaluation.copy()
        aval.sort(key=lambda individual: individual.aval_norm)
        
        #contrafactual_ind = pd.DataFrame(columns=self.input_dataset.columns)
        solution_list = []
        solution_colums_list = []
        
        i = 0
        numContraf = 0
        while i < len(aval) and numContraf <= num_elit + 1:
            #Checks if the example belongs to the counterfactual class
            if aval[i].predict_proba < 0.5:
                ind_changes = []
                ind_colums_change = []
         
                #Gets counterfactual example change list
                ind_changes = self.getChanges(aval[i].index, parents)
                #Generates the list of columns modified in the counterfactual to check if there is already a solution with that set of columns
                ind_colums_change = self.getColumns(ind_changes)
                
                if ind_colums_change not in solution_colums_list:
                    #Check if one solution is a subset of the other
                    if not self.contained_solution(self.original_ind, solution_list, solution_colums_list, ind_changes, ind_colums_change):
                        #Include counterfactual in the list of examples of the final solution                    
                        df.loc[len(df)] = parents.iloc[aval[i].index].copy()                     
                                
                        #Add to the list of solutions (changes only)       
                        solution_list.append(ind_changes)
                        #Used to compare with the next counterfactuals (to ensure diversity)
                        solution_colums_list.append(ind_colums_change)
                                        
                        numContraf = numContraf + 1
                      
            i = i + 1
        return solution_list
    
    def roulette_wheel(self, evaluation):
        summation = 0
        #Performs roulette wheel to select parents who will undergo genetic operations
        for i in range (1, len(evaluation)): 
            summation = summation + 1/evaluation[i].aval_norm
    
        roulette = rnd.uniform( 0, summation )
    
        roulette_score = 1/evaluation[1].aval_norm
        i = 1
        while roulette_score < roulette:
            i += 1
            roulette_score += 1/evaluation[i].aval_norm
        
        return i
            
    def crossover (self, df, parents, evaluation, number_cross_repetitions):
        child = []
            
        corte = rnd.randint( 0, self.input_dataset.columns.size - 1 )
            
        index1 = self.roulette_wheel(evaluation)
        index2 = self.roulette_wheel(evaluation)
        
        ind_a = parents.iloc[index1].copy()
        ind_b = parents.iloc[index2].copy()
            
        crossover_op = rnd.random()
        if crossover_op <= self.cros_proba:
            child[ :corte ] = ind_a[ :corte ].copy()
            child[ corte: ] = ind_b[ corte: ].copy()
        else:
            child = ind_a.copy()
        
        ni = self.equal(child, df)
        if ni == 0:
            df.loc[len(df)] = child.copy()
        else:
            #Assesses whether the GA is producing too many repeated individuals.
            number_cross_repetitions = number_cross_repetitions + 1
            if number_cross_repetitions == self.pop_size:
                self.pop_size = round(self.pop_size - self.pop_size*0.1)
                self.mutation_proba = self.mutation_proba + 0.1
                #print('Adjusting population size...', self.pop_size)
                number_cross_repetitions = 0
        #    print('repeated')
        return number_cross_repetitions
                       
    def mutation (self, df, individual_pos, features_range):
        ni = 1
        #Does not allow repeated individual
        while ni == 1:
            #Draw a feature to change
            index_a = rnd.randint( 0, self.input_dataset.columns.size - 1 )
            while df.columns[index_a] in self.static_list:
                index_a = rnd.randint( 0, self.input_dataset.columns.size - 1 )
            
            if not features_range[index_a].unique_value():
                #Mutation
                mutant = df.iloc[individual_pos].copy()
            
                #Draw the value to be changed
                new_value =  self.getMutationValue(mutant.iloc[index_a], index_a, features_range[index_a])  
                mutant.iloc[index_a] = new_value

                ni = self.equal(mutant, df)
                if ni == 0:
                    df.loc[individual_pos] = mutant.copy()
                #else:
                #    print('repeated')
     
    def getChanges(self, ind, dfComp):
        changes = []
        
        for i in range (len(dfComp.iloc[ind])):
            if self.original_ind[i] != dfComp.loc[ind][i]:
                counter_change_ind = counter_change(dfComp.columns[i], dfComp.loc[ind][i])
                changes.append(counter_change_ind)

        return changes
    
    #Generates the solution from the final population
    def getContrafactual(self, df, aval):
        
        contrafactual_ind = pd.DataFrame(columns=self.input_dataset.columns)
        solution_list = []
        solution_colums_list = []
        
        i = 0
        numContraf = 0
        while i < len(aval) and numContraf < self.K:
            #Checks if the example belongs to the counterfactual class
            if aval[i].predict_proba < 0.5:
                ind_changes = []
                ind_colums_change = []
         
                #Gets counterfactual example change list
                ind_changes = self.getChanges(aval[i].index, df)
                #Generates the list of columns modified in the counterfactual to check if there is already a solution with that set of columns
                ind_colums_change = self.getColumns(ind_changes)
                
                if ind_colums_change not in solution_colums_list:
                    #Check if one solution is a subset of the other
                    if not self.contained_solution(self.original_ind, solution_list, solution_colums_list, ind_changes, ind_colums_change):
                        #Include counterfactual in the list of examples of the final solution
                        contrafactual_ind.loc[len(contrafactual_ind)] = df.iloc[aval[i].index].copy()
                                
                        #Add to the list of solutions (changes only)       
                        solution_list.append(ind_changes)
                        #Used to compare with the next counterfactuals (to ensure diversity)
                        solution_colums_list.append(ind_colums_change)
                                        
                        numContraf = numContraf + 1
                        #print('solution_list ', solution_list)
                    #else:
                        #print('is contained ', ind_changes)
                #else:
                    #print('repeated ', ind_changes)
                      
            i = i + 1

        return contrafactual_ind, solution_list   
    
    def printResults(self, solution):
        print("Result obtained")
        if len(solution) != 0:
            for i in range(0, len(solution)): 
                print("\n")
                print(f"{'Counterfactual ' + str(i + 1):^34}")
                for j in range(0, len(solution[i])): 
                    print(f"{str(solution[i][j].column):<29} {str(solution[i][j].value):>5}")
        else:
            print('Solution not found. It may be necessary to adjust the parameters for this instance.')
                                                 
    def explain(self, original_ind, current_class):
        self.original_ind = original_ind #Original instance
        #self.ind_cur_class = ind_cur_class #Index in the shap corresponds to the original instance class
        self.current_class = current_class #Original instance class
        
        ind_cur_class = self.getBadClass()
    
        #Gets the valid values range of each feature
        features_range = []
        features_range = self.getFeaturesRange()

        #The DataFrame df will have the current population
        df = pd.DataFrame(columns=self.input_dataset.columns)
        
        #Generates the initial population with popinitial mutants        
        self.getPopInicial(df, features_range)
        for g in range(self.num_gen):
            #To use on the parents of each generation
            parents = pd.DataFrame(columns=self.input_dataset.columns)
    
            #Copy parents to the next generation
            parents = df.copy()
            #df will contain the new population
            df = pd.DataFrame(columns=self.input_dataset.columns)
            
            evaluation = []                         
                   
            #Assessing generation counterfactuals
            self.fitness(parents, evaluation, ind_cur_class)
            #The original individual will always be in the 0 position of the df - So that it is normalized too (it will be used later in the distance function)
            df.loc[0] = self.original_ind.copy()
            
            #Copies to the next generation the per_elit best individuals
            self.elitism(evaluation, df, parents)
            number_cross_repetitions = 0
            while len(df) < self.pop_size + 1: #+1, as the 1st position is used to store the reference individual
                number_cross_repetitions = self.crossover(df, parents, evaluation, number_cross_repetitions)
                
                mutation_op = rnd.random()
                if mutation_op <= self.mutation_proba:
                    self.mutation(df, len(df) - 1, features_range)
            
            print()
                 
        evaluation = []
    
        #Evaluating the latest generation
        self.fitness(df, evaluation, ind_cur_class)
    
        #Order the last generation by distance to the original instance     
        evaluation.sort(key=lambda individual: individual.aval_norm)     
        
        #Getting the counterfactual set
        contrafactual_set = pd.DataFrame(columns=self.input_dataset.columns)
        contrafactual_set, solution_list = self.getContrafactual(df, evaluation)       
                 
        return contrafactual_set, solution_list
    
    
    




class CCSSE:
    def __init__(self, dataset, bb_model, dataset_size, samples = None, K = 5, generation = 10):
        self.df_datasets = self.load_df_dataset()
        self.dataset = dataset
        self.samples = samples
        self.K = K
        self.generation = generation
        self.dataset_size = dataset_size
        
#         self.x_train, self.x_test, self.y_train, self.y_test, self.dfx_full, self.dfy_full = self.get_datasets_train_test()
        self.x_train, self.x_test, self.y_train, self.y_test, self.dfx_full, self.dfy_full = self.get_dataset()
    
        print(f"Tamanho do x_train = {self.x_train.shape}")
        print(f"Tamanho do x_test = {self.x_test.shape}")
        print(f"Tamanho do y_train = {self.y_train.shape}")
        print(f"Tamanho do y_test = {self.y_test.shape}")
        print(f"Tamanho do dfx_full = {self.dfx_full.shape}")
        print(f"Tamanho do dfy_full = {self.dfy_full.shape}")

        self.bb_model, self.p = self.get_bb_model(bb_model)
        self.explainerCSSE = self.get_model_contrafactual()
        self.model_causal, self.df_causal_effects, self.df_error, self.causal_order = self.get_model_causality()
    
        self.run_dict = {}
        self.run_non_causal_dict = {}
        
    def load_df_dataset(self):
        def convert_to_list(val):
            try:
                return ast.literal_eval(val) if isinstance(val, str) and val.startswith('[') and val.endswith(']') else val
            except (ValueError, SyntaxError):
                return val
            
        df = pd.read_parquet("s3://omar-testes-gerais/artigos/artifacts/dfm_use.parquet")
        df['path'] = df['path'].apply(convert_to_list)
        
        return df


    def get_dataset(self):
        dataset_dict = self.df_datasets[self.df_datasets['name'] == self.dataset].iloc[0].to_dict()
        
        if isinstance(dataset_dict['path'], list):
            if 'Column' in dataset_dict['classe']:
                df_train = pd.read_csv(f"s3://omar-testes-gerais/artigos/artifacts/datasets/{dataset_dict['path'][0]}", header = None, nrows=self.dataset_size)
                df_train.columns = df_train.columns.astype(str)
                df_test = pd.read_csv(f"s3://omar-testes-gerais/artigos/artifacts/datasets/{dataset_dict['path'][1]}", header = None, nrows=self.dataset_size)
                df_test.columns = df_test.columns.astype(str)
                class_name = str(int(dataset_dict['class'].split('Column')[1]) - 1)
            else:
                df_train = pd.read_csv(f"s3://omar-testes-gerais/artigos/artifacts/datasets/{dataset_dict['path'][0]}", nrows=self.dataset_size)
                df_test = pd.read_csv(f"s3://omar-testes-gerais/artigos/artifacts/datasets/{dataset_dict['path'][1]}", nrows=self.dataset_size)
                class_name = dataset_dict['classe']
            
            x_train = df_train.drop(columns=[class_name])
            y_train = df_train[class_name]

            # Dividindo o df_test
            x_test = df_test.drop(columns=[class_name])
            y_test = df_test[class_name]
            
            dfx_full = pd.concat([x_train, x_test])
            dfy_full = pd.concat([y_train, y_test])
            
        
        else:
            if 'Column' in dataset_dict['classe']:
                df_main = pd.read_csv(f"s3://omar-testes-gerais/artigos/artifacts/datasets/{dataset_dict['path']}", header = None, nrows=self.dataset_size)
                df_main.columns = df_main.columns.astype(str)
                class_name = str(int(dataset_dict['classe'].split('Column')[1]) - 1)
            else:
                df_main = pd.read_csv(f"s3://omar-testes-gerais/artigos/artifacts/datasets/{dataset_dict['path']}", nrows=self.dataset_size)
                class_name = dataset_dict['classe']
            
            columns = df_main.columns
            columns_tmp = list(columns)
            columns_tmp.remove(class_name)

            x_train, x_test, y_train, y_test = train_test_split(df_main[columns_tmp], df_main[class_name], test_size=0.1)

            dfx_full = pd.concat([x_train, x_test])
            dfy_full = pd.concat([y_train, y_test])
            
        return x_train, x_test, y_train, y_test, dfx_full, dfy_full
    
    def get_datasets_train_test(self):
        if self.dataset == "german_short":
            map_columns = {
                'Unnamed: 0': 'index',
                'x0': 'Sex',
                'x1': 'Age',
                'x2': 'Credit',
                'x3': 'LoanDuration',
            }
            x_train = pd.read_csv("data/algrec_german/X_train_short.csv").rename(columns = map_columns)
            x_test = pd.read_csv("data/algrec_german/X_test_short.csv").rename(columns = map_columns)
            y_train = pd.read_csv("data/algrec_german/y_train_short.csv").rename(columns={'Unnamed: 0': 'index'})
            y_test = pd.read_csv("data/algrec_german/y_test_short.csv").rename(columns={'Unnamed: 0': 'index'})
            x_train = x_train.set_index('index')
            x_test = x_test.set_index('index')
            y_train = y_train.set_index('index')
            y_test = y_test.set_index('index')
            dfx_full = pd.concat([x_train, x_test])
            dfy_full = pd.concat([y_train, y_test])

        elif self.dataset == "german_medium":
            map_columns = {
                'Unnamed: 0': 'index',
                'x0': 'Sex',
                'x1': 'Age',
                'x2': 'Credit',
                'x3': 'LoanDuration',
                'x4': 'CheckingAccountBalance',
                'x5':'SavingsAccountBalance',
                'x6':'HousingStatus'
            }
            x_train = pd.read_csv("data/algrec_german/X_train.csv").rename(columns = map_columns)
            x_test = pd.read_csv("data/algrec_german/X_test.csv").rename(columns = map_columns)
            y_train = pd.read_csv("data/algrec_german/y_train.csv").rename(columns={'Unnamed: 0': 'index'})
            y_test = pd.read_csv("data/algrec_german/y_test.csv").rename(columns={'Unnamed: 0': 'index'})
            x_train = x_train.set_index('index')
            x_test = x_test.set_index('index')
            y_train = y_train.set_index('index')
            y_test = y_test.set_index('index')
            dfx_full = pd.concat([x_train, x_test])
            dfy_full = pd.concat([y_train, y_test])

        elif self.dataset == "german_full":
            df_main = prepare_german_dataset("german_credit.csv", "data/")
            columns = df_main.columns
            class_name = 'default' # default = 0 = "Good class" / default = 1 = "Bad class" 
            columns_tmp = list(columns)
            columns_tmp.remove(class_name)

            x_train, x_test, y_train, y_test = train_test_split(df_main[columns_tmp], df_main[class_name], test_size=0.1)

            dfx_full = pd.concat([x_train, x_test])
            dfy_full = pd.concat([y_train, y_test])
            
        elif self.dataset == 'np':
            df_main = pd.read_csv("data/breast_coimbra_np.csv")
            columns = df_main.columns
            class_name = 'Classification' # default = 0 = "Good class" / default = 1 = "Bad class" 
            columns_tmp = list(columns)
            columns_tmp.remove(class_name)

            x_train, x_test, y_train, y_test = train_test_split(df_main[columns_tmp], df_main[class_name], test_size=0.1)

            dfx_full = pd.concat([x_train, x_test])
            dfy_full = pd.concat([y_train, y_test])
            
        elif self.dataset == 'nm':
            df_main = pd.read_csv("data/heloc_dataset_v1_nm.csv")
            columns = df_main.columns
            class_name = 'RiskPerformance' # default = 0 = "Good class" / default = 1 = "Bad class" 
            columns_tmp = list(columns)
            columns_tmp.remove(class_name)

            x_train, x_test, y_train, y_test = train_test_split(df_main[columns_tmp], df_main[class_name], test_size=0.1)

            dfx_full = pd.concat([x_train, x_test])
            dfy_full = pd.concat([y_train, y_test])
            
        elif self.dataset == 'cm':
            df_main = pd.read_csv("data/house_votes_84_processada_cm.csv")
            columns = df_main.columns
            class_name = 'Class Name' # default = 0 = "Good class" / default = 1 = "Bad class" 
            columns_tmp = list(columns)
            columns_tmp.remove(class_name)

            x_train, x_test, y_train, y_test = train_test_split(df_main[columns_tmp], df_main[class_name], test_size=0.1)

            dfx_full = pd.concat([x_train, x_test])
            dfy_full = pd.concat([y_train, y_test])
            
        elif self.dataset == 'ng':
            df_main = pd.read_csv("data/ionosphere_ng.csv")
            columns = df_main.columns
            class_name = 'target' # default = 0 = "Good class" / default = 1 = "Bad class" 
            columns_tmp = list(columns)
            columns_tmp.remove(class_name)

            x_train, x_test, y_train, y_test = train_test_split(df_main[columns_tmp], df_main[class_name], test_size=0.1)

            dfx_full = pd.concat([x_train, x_test])
            dfy_full = pd.concat([y_train, y_test])
            
        elif self.dataset == 'tokyo_ng':
            df_main = pd.read_csv("data/Tokyo_ng.csv").rename(columns = {'Unnamed: 44': 'class'})
            columns = df_main.columns
            class_name = 'class' # default = 0 = "Good class" / default = 1 = "Bad class" 
            columns_tmp = list(columns)
            columns_tmp.remove(class_name)

            x_train, x_test, y_train, y_test = train_test_split(df_main[columns_tmp], df_main[class_name], test_size=0.1)

            dfx_full = pd.concat([x_train, x_test])
            dfy_full = pd.concat([y_train, y_test])
            
        elif self.dataset == 'breast-cancer_ng':
            df_main = pd.read_csv("data/breast-cancer_ng.csv")
            columns = df_main.columns
            class_name = 'diagnosis' # default = 0 = "Good class" / default = 1 = "Bad class" 
            columns_tmp = list(columns)
            columns_tmp.remove(class_name)

            x_train, x_test, y_train, y_test = train_test_split(df_main[columns_tmp], df_main[class_name], test_size=0.1)

            dfx_full = pd.concat([x_train, x_test])
            dfy_full = pd.concat([y_train, y_test])
        
        elif self.dataset == 'cp':
            df_train = pd.read_csv("data/monks-1_train_cp.csv")

            #Get the input features
            columns = df_train.columns
            class_name = 'Class' # default = 0 = "Good class" / default = 1 = "Bad class" 
            columns_tmp = list(columns)
            columns_tmp.remove(class_name)
            columns_tmp.remove('Id')

            x_train = df_train[columns_tmp]
            y_train = df_train[['Class']]

            # x_train, x_test, y_train, y_test = train_test_split(df_main[columns_tmp], df_main[class_name], test_size=0.1)

            model = RandomForestClassifier()  
            model.fit(x_train, y_train)

            df_test = pd.read_csv('data/monks-1_test_cp.csv')

            #Get the input features
            columns = df_test.columns
            class_name = 'Class' # default = 0 = "Good class" / default = 1 = "Bad class" 
            columns_tmp = list(columns)
            columns_tmp.remove(class_name)
            columns_tmp.remove('Id')

            x_test = df_test[columns_tmp]
            y_test = df_test[['Class']]

            dfx_full = pd.concat([x_train, x_test])
            dfy_full = pd.concat([y_train, y_test])
            
        elif self.dataset == 'cg':
            df_main = pd.read_csv("data/mushrooms_processada_cg.csv")[:1000]
            columns = df_main.columns
            class_name = 'class' # default = 0 = "Good class" / default = 1 = "Bad class" 
            columns_tmp = list(columns)
            columns_tmp.remove(class_name)

            x_train, x_test, y_train, y_test = train_test_split(df_main[columns_tmp], df_main[class_name], test_size=0.1)

            dfx_full = pd.concat([x_train, x_test])
            dfy_full = pd.concat([y_train, y_test])
            
        elif self.dataset == 'Phishing_cg':
            df_main = pd.read_csv("data/Phishing_cg.csv")[2:1000].astype(float)
            columns = df_main.columns
            class_name = 'Class' # default = 0 = "Good class" / default = 1 = "Bad class" 
            columns_tmp = list(columns)
            columns_tmp.remove(class_name)

            x_train, x_test, y_train, y_test = train_test_split(df_main[columns_tmp], df_main[class_name], test_size=0.1)

            dfx_full = pd.concat([x_train, x_test])
            dfy_full = pd.concat([y_train, y_test])

        else:
            x_train = pd.DataFrame()
            x_test = pd.DataFrame()
            y_train = pd.DataFrame()
            y_test = pd.DataFrame()
            dfx_full = pd.DataFrame()
            dfy_full = pd.DataFrame()

        return x_train, x_test, y_train, y_test, dfx_full, dfy_full
    

    def get_bb_model(self, bb_model_name):
        
        if bb_model_name == 'rf':
            bb_model = RandomForestClassifier()  
            bb_model.fit(self.x_train, self.y_train)

            p = bb_model.predict(self.x_test)

            print(classification_report(self.y_test, p))

            return bb_model, p
        elif bb_model_name == 'rn':
            bb_model = MLPClassifier()  
            bb_model.fit(self.x_train, self.y_train)

            p = bb_model.predict(self.x_test)

            print(classification_report(self.y_test, p))

            return bb_model, p

    def get_model_contrafactual(self):
        return CSSE(self.dfx_full, self.bb_model, K = self.K, num_gen = self.generation)

    def get_model_causality(self):
        model_causal = lingam.DirectLiNGAM()
        model_causal.fit(self.dfx_full)
            
        labels = [f'{i}' for i in self.dfx_full.columns]
        causal_order = [labels[x] for x in model_causal.causal_order_]
        
        matrix = model_causal.adjacency_matrix_
        from_list = []
        to_list = []
        effect_list = []

        # Iteração sobre a matriz para extrair os valores e suas posições
        for i in range(len(matrix)):
            for j in range(len(matrix[i])):
                if matrix[i][j] != 0:
                    from_list.append(j)
                    to_list.append(i)
                    effect_list.append(matrix[i][j])

        # Criando o DataFrame
        df_causal_effects = pd.DataFrame({'from': from_list, 'to': to_list, 'effect': effect_list})
        labels = [f'{i}' for i in self.dfx_full.columns]
        df_causal_effects['from'] = df_causal_effects['from'].apply(lambda x : labels[x])
        df_causal_effects['to'] = df_causal_effects['to'].apply(lambda x : labels[x])


        matrix_error = model_causal.get_error_independence_p_values(self.dfx_full)
        from_list = []
        to_list = []
        effect_list = []

        # Iteração sobre a matriz para extrair os valores e suas posições
        for i in range(len(matrix_error)):
            for j in range(i + 1, len(matrix_error[i])):
                if matrix_error[i][j] != 0:
                    from_list.append(j)
                    to_list.append(i)
                    effect_list.append(matrix_error[i][j])

        # Criando o DataFrame
        df_error = pd.DataFrame({'from': from_list, 'to': to_list, 'effect': effect_list})
        labels = [f'{i}' for i in self.dfx_full.columns]
        df_error['from'] = df_error['from'].apply(lambda x : labels[x])
        df_error['to'] = df_error['to'].apply(lambda x : labels[x])
        df_error.fillna(0, inplace = True)
        

        return model_causal, df_causal_effects, df_error, causal_order
        
    
    def print_causal_graph(self):
        make_dot(self.model_causal.adjacency_matrix_)

    def run_non_causal(self):
        self.run_non_causal_dict = {}

        if isinstance(self.samples, list):
            self.create_run_dict(self)
            for sample in self.samples:
                self.run_non_causal_sample(sample)
                
        elif isinstance(self.samples, int):
            for sample in range(self.samples):
                self.run_non_causal_sample(sample)
        
        else: 
            for sample in range(10):
                self.run_non_causal_sample(sample)
                
    def run_non_causal_sample(self, sample):
        self.run_non_causal_dict[sample] = {}
        original_instance = self.x_test.iloc[sample].copy()
        contrafactual_set, solution = self.explainerCSSE.explain(original_instance, self.p[sample]) #Method returns the list of counterfactuals and the explanations generated from them

        self.run_non_causal_dict[sample]['solution'] = solution

    def run_causal(self):
        start_time = time.time()
        self.run_dict = {}
        self.run_dict['global_numbers'] = {
                    "global_quant_changes": 0,
                    "global_quant_causal_changes": 0,
                    "global_quant_causal_rules": 0,
                    "global_quant_zeros_causal": 0,
                    "global_quant_full_causal": 0,
                    "global_quant_causal_contrafac": 0,
                    "global_quant_maioria_causal_satisfeita": 0,
                    "global_quant_contrafac_unico": 0,
            }
        self.global_quant_contrafac_max = 0
        if isinstance(self.samples, list):
            for sample in self.samples:
                self.run_causal_sample(sample)
                
        elif isinstance(self.samples, int):
            for sample in range(self.samples):
                try:
                    self.run_causal_sample(sample)
                except Exception as e:
                    print(f"DEBUG ERRO: {e}")
        
        else: 
            for sample in range(10):
                self.run_causal_sample(sample)
        
        self.global_quant_contrafac_max = self.K * len(self.run_dict)
        self.run_dict['global_numbers']['global_timing_run_causal'] = time.time() - start_time


    def run_causal_sample(self, sample):
        if isinstance(self.samples, list):
            original_instance = self.dfx_full.iloc[sample]
        else:
            original_instance = self.x_test.iloc[sample]
        self.run_dict[sample] = {}
        self.run_dict[sample]['original_instance'] = original_instance

#         print(f'Running original instance:\n {display(original_instance)}')
        print(f'Start to Running samples')

        causal_explain = self.get_causal_explain(sample)
        self.run_dict[sample]['causal_explain'] = causal_explain

        list_analyse = []
        for contrafactual in causal_explain[0]:
            list_analyse.append(self.analyse_contrafac(contrafactual, causal_explain[1], causal_explain[2]))

        self.run_dict[sample]['list_analyse'] = list_analyse
        self.analyse_explaination(sample)

    def analyse_contrafac(self, contrafac, df, original_ind):
        columns = [x.column for x in contrafac]
        condicao = (df['to'].isin(columns)) & (df['from'].isin(columns))
        ind = original_ind[columns]
        return [contrafac, df[condicao], ind]

    def get_causal_explain(self, sample):
        if isinstance(self.samples, list):
            original_ind = self.dfx_full.iloc[sample].copy()
        else:
            original_ind = self.x_test.iloc[sample].copy() #Original instance
        #self.ind_cur_class = ind_cur_class #Index in the shap corresponds to the original instance class
        self.explainerCSSE.current_class = self.p[sample] #Original instance class
        self.explainerCSSE.original_ind = original_ind
        
        ind_cur_class = self.explainerCSSE.getBadClass()

        #Gets the valid values range of each feature
        features_range = []
        features_range = self.explainerCSSE.getFeaturesRange()

        #The DataFrame df will have the current population
        df = pd.DataFrame(columns=self.explainerCSSE.input_dataset.columns)

        #Generates the initial population with popinitial mutants        
        self.explainerCSSE.getPopInicial(df, features_range)
        global df_causal
        df_causal = df.copy()
        global dict_dfs
        dict_dfs = {}
        

        # for g in tqdm(range(self.explainerCSSE.num_gen), desc= "Processing..."):
        for g in range(self.generation):
            print(f"Generation = {g}")
            #To use on the parents of each generation
            global old_parents
            old_parents = pd.DataFrame(columns=self.explainerCSSE.input_dataset.columns)
            #Copy parents to the next generation
            old_parents = df_causal.copy()
            dict_dfs[g] = {}

            parents_causal = self.apply_causality(old_parents)
            dict_dfs[g]['causal_parents'] = parents_causal
            
            #df will contain the new population
            df_causal = pd.DataFrame(columns=self.explainerCSSE.input_dataset.columns)
            evaluation_causal = []

            #Assessing generation counterfactuals
            self.explainerCSSE.fitness(dict_dfs[g]['causal_parents'], evaluation_causal, ind_cur_class)

            #The original individual will always be in the 0 position of the df - So that it is normalized too (it will be used later in the distance function)
            df_causal.loc[0] = original_ind.copy()

            #Copies to the next generation the per_elit best individuals
            self.explainerCSSE.elitism(evaluation_causal, df_causal, parents_causal)
            number_cross_repetitions = 0
            while len(df_causal) < self.explainerCSSE.pop_size + 1: #+1, as the 1st position is used to store the reference individual
                number_cross_repetitions_causal = self.explainerCSSE.crossover(df_causal, parents_causal, evaluation_causal, number_cross_repetitions)

                mutation_op = rnd.random()
                if mutation_op <= self.explainerCSSE.mutation_proba:
                    self.explainerCSSE.mutation(df_causal, len(df_causal) - 1, features_range)


        evaluation = []
        evaluation_causal = []

        #Evaluating the latest generation
        self.explainerCSSE.fitness(df_causal, evaluation_causal, ind_cur_class)

        #Order the last generation by distance to the original instance     
        evaluation_causal.sort(key=lambda individual: individual.aval_norm) 

        #Getting the counterfactual CAUSAL set
        contrafactual_set_causal, solution_list_causal = self.explainerCSSE.getContrafactual(df_causal, evaluation_causal) 

        dict_dfs['contrafactual_set_causal'] = contrafactual_set_causal
        dict_dfs['solution_list_causal'] = solution_list_causal
        
        df_contrafac_causal = self.get_contrafac_df_causal(solution_list_causal)
        return [solution_list_causal, df_contrafac_causal, original_ind]
    

    def apply_causality(self, df):
        count_print = 0
        df_apply_causal = pd.DataFrame(columns = df.columns)
        original = df.iloc[0]
        df_apply_causal.loc[0] = original
        for index, df_row in df.iloc[1:].iterrows():
#             print(f"index old_parent = {index}")
            causal_ind = df_row.copy()
#             for column_name in causal_ind.index:
#                 print(f"{column_name} = {causal_ind[column_name]}")
#             print()
            for column in self.causal_order:
                value_diff = causal_ind[column] - original[column]
                if value_diff != 0:
                    tmp_effects = self.df_causal_effects[self.df_causal_effects['from'] == column]
                    for index, row in tmp_effects.iterrows():
                        
    #                     prob = rnd.random()
    #                     if row['probability'] <= prob:
                        tmp_error = self.df_error[self.df_error['from'].isin([column, row['to']]) | self.df_error['to'].isin([column, row['to']])]
                        error_value = tmp_error['effect'].iloc[0]
    #                     print(f'error value = {error_value}')
                        new_value = causal_ind[row['to']] + (value_diff * row['effect']) + tmp_error['effect'].iloc[0]
#                         if np.isnan(new_value) and count_print == 0:
#                             print(f"row['to'] = {row['to']}")
#                             print(f"causal_ind[row['to']] = {causal_ind[row['to']]}")
#                             print(f"new_value = {new_value}")
#                             print(f"value_diff = {value_diff}")
#                             print(f"row['effect'] = {row['effect']}")
#                             print(f"tmp_error['effect'].iloc[0] = {tmp_error['effect'].iloc[0]}")
#                             count_print = 1
                        causal_ind[row['to']] = new_value
            df_apply_causal.loc[len(df_apply_causal)] = causal_ind
        return df_apply_causal


    def get_contrafac_df_causal(self, solution_list_causal):
        lista_solution_causal = [[t.column for t in sublist] for sublist in solution_list_causal]

        # Inicializa uma lista para armazenar os resultados
        resultados = []

        # Loop sobre os valores na lista
        for lista_valores in lista_solution_causal:
            if len(lista_valores) > 1:
                for v1 in lista_valores:
                    for v2 in lista_valores:
                        if v1 != v2:
                            # Cria uma condição para cada par de valores diferentes na lista
                            condicao = (self.df_causal_effects['to'].isin([v1, v2])) & (self.df_causal_effects['from'].isin([v1, v2]))
                            # Realiza a busca no DataFrame usando a condição e armazena os resultados
                            resultados.append(self.df_causal_effects[condicao])

        # Concatena os resultados em um único DataFrame
        if resultados:
            resultado_final = pd.concat(resultados)
            resultado_final = resultado_final.drop_duplicates()
        else:
            resultado_final = pd.DataFrame(columns = self.df_causal_effects.columns)
            
        return resultado_final
    

    def analyse_explaination(self, sample):
        self.run_dict[sample]['data_analysis'] = []
        for i, content in enumerate(self.run_dict[sample]['list_analyse']):
            self.global_quant_contrafac_max += 1
            controle = {}
            causal = content[0]
            df = content[1]
            ori = content[2]
            
            
            num_changes = len(causal)
            self.run_dict['global_numbers']['global_quant_changes'] += num_changes
            
            num_causal_rules = len(df)
            self.run_dict['global_numbers']['global_quant_causal_rules'] += num_causal_rules
            
            for attr in causal:
                key = attr.column
                if attr.value > ori[key]:
                    controle[key] = 'mais'
                else:
                    controle[key] = 'menos'

            df_temp = df.copy()
            df_temp['from'] = df['from'].map(controle)
            df_temp['to'] = df['to'].map(controle)
            if len(df_temp) > 0:
                df_temp['causal'] = df_temp.apply(lambda row: self.verificar_condicoes(row), axis = 1)
                causal_finds = df_temp['causal'].sum()
            else:
                causal_finds = 0
                
            data_dict = {}

            data_dict['df_respeita_causal'] = df_temp
            data_dict['contrafactual_causal'] = causal
            data_dict['df_causal_effects'] = df
            
            self.run_dict[sample]['data_analysis'].append(data_dict)

            self.run_dict['global_numbers']['global_quant_causal_changes'] += causal_finds
            
            # print(f'causal = \n{causal}\n')
            # print(f'original = \n{ori}\n')
            # print(f'df_temp = \n{display(df_temp)}\n')
            
            if len(df_temp) > 0:
                if causal_finds > 0:
                    self.run_dict['global_numbers']['global_quant_causal_contrafac'] += 1
                else:
                    # print(f'nenhuma relaçao causal satisfeita')
                    self.run_dict['global_numbers']['global_quant_zeros_causal'] += 1
    #                 display(df_temp)
    #                 print(f"original = {ori}")
    #                 print(f"causal = {causal}")

                if causal_finds == num_causal_rules:
                    self.run_dict['global_numbers']['global_quant_full_causal'] += 1
                    # if causal_finds > 2:
                        # print(f'todas > 2 relaçoes causais satisfeitas')
    #                     display(df_temp)
    #                     print(f"original = {ori}")
    #                     print(f"causal = {causal}")
                    # elif causal_finds == 1:
                        # print(f'todas = 1 relaçoes causais satisfeitas')
                
                if causal_finds >= (len(df_temp)/2):
                    self.run_dict['global_numbers']['global_quant_maioria_causal_satisfeita'] += 1
            else:
    #             if len(causal) > 0:
                self.run_dict['global_numbers']['global_quant_contrafac_unico'] += 1
        
    def verificar_condicoes(self, row):
        if (row['from'] == 'mais' and row['to'] == 'mais' and row['effect'] > 0):
            return True
        elif row['from'] == 'menos' and row['to'] == 'menos' and row['effect'] > 0:
            return True
        elif row['from'] == 'mais' and row['to'] == 'menos' and row['effect'] < 0:
            return True
        elif row['from'] == 'menos' and row['to'] == 'mais' and row['effect'] < 0:
            return True
        else:
            return False
    
            

    def show_metrics(self, get_output = False):
        
        print(f"Quantidade de instâncias contrafactuais = {self.global_quant_contrafac_max}")
        print(f'Quantidade de relações causais na base de dados = {len(self.df_causal_effects)}')
        print(f"Quantidade de atributos modificados = {self.run_dict['global_numbers']['global_quant_changes']}")
        print(f"Quantidade de instâncias contrafactuais causais = {self.run_dict['global_numbers']['global_quant_contrafac_unico'] + self.run_dict['global_numbers']['global_quant_causal_contrafac']}")
        print(f"Quantidade de relações causais analisadas = {self.run_dict['global_numbers']['global_quant_causal_rules']}")
        print(f"Quantidade de relações causais satisfeitas = {self.run_dict['global_numbers']['global_quant_causal_changes']}")
        print(f"Quantidade de instâncias contrafactuais com um único atributo modificado = {self.run_dict['global_numbers']['global_quant_contrafac_unico']}")
        print(f"Tempo de execução = {self.run_dict['global_numbers']['global_timing_run_causal']}")
        
        if get_output:
            metrics_dict = {
                "Quantidade de instâncias contrafactuais": self.global_quant_contrafac_max,
                "Quantidade de relações causais na base de dados": len(self.df_causal_effects),
                "Quantidade de atributos modificados": self.run_dict['global_numbers']['global_quant_changes'],
                "Quantidade de instâncias contrafactuais causais": self.run_dict['global_numbers']['global_quant_contrafac_unico'] + self.run_dict['global_numbers']['global_quant_causal_contrafac'],
                "Quantidade de relações causais analisadas": self.run_dict['global_numbers']['global_quant_causal_rules'],
                "Quantidade de relações causais satisfeitas": self.run_dict['global_numbers']['global_quant_causal_changes'],
                "Quantidade de instâncias contrafactuais com um único atributo modificado": self.run_dict['global_numbers']['global_quant_contrafac_unico'],
                "Tempo de execução": self.run_dict['global_numbers']['global_timing_run_causal']
            }
        
            return metrics_dict

def get_causal_metrics(row, bb_model_name, dataset_size):
    print(row)
    global ccsse
    ccsse = CCSSE(row['name'], K = 10, generation= 10, bb_model = bb_model_name, dataset_size = dataset_size) #, samples = 10
    print(f"criou o modelo")
    ccsse.run_causal()
    print(f"run_causal completo")
    dict_metricas = ccsse.show_metrics(get_output = True)
    converted_dict_metricas = convert_np_types(dict_metricas)
    json_data = json.dumps(converted_dict_metricas, indent=4)

    print(f"Execução completa para {row['name']}")
    return json_data

def convert_np_types(data):
    """Converte tipos de dados NumPy em tipos nativos do Python."""
    if isinstance(data, dict):
        return {key: convert_np_types(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [convert_np_types(item) for item in data]
    elif isinstance(data, np.int64):
        return int(data)  # Converte int64 para int
    elif isinstance(data, np.float64):
        return float(data)  # Converte float64 para float
    else:
        return data

#PJ LOCAL
def handler(dataset, model_name, dataset_size):
    
    args = {
        "list_dataset_name": dataset,
        'bb_model_name': model_name,
        'dataset_size': dataset_size
    }
    
    dfm_use = pd.read_parquet(f"s3://omar-testes-gerais/artigos/artifacts/dfm_use.parquet")

    for idx, row in dfm_use.iterrows():
        if row['name'] in args['list_dataset_name']:
            json_response = get_causal_metrics(row, args["bb_model_name"], dataset_size = args["dataset_size"])
            return json_response
    
    return dfm_use

#PJ REAL
# if __name__ == '__main__':
    
#     parser = argparse.ArgumentParser()
    
#     # Adicionando os argumentos para input e output
#     parser.add_argument('--list_dataset_name', type=str)
#     parser.add_argument('--bb_model_name', type=str)
    
#     args = parser.parse_args()
#     list_dataset_name = ast.literal_eval(args.list_dataset_name)

#     df_map_inference_datasets = pd.read_parquet(f"s3://omar-testes-gerais/artigos/artifacts/df_map_inference_datasets.parquet")
#     df_dataset = df_map_inference_datasets[df_map_inference_datasets['name'].isin(list_dataset_name)]
#     df_dataset.apply(lambda x: get_causal_metrics(x, args.bb_model_name), axis = 1)


  Using cached lingam-1.9.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached graphviz-0.20.3-py3-none-any.whl.metadata (12 kB)
  Using cached pygam-0.9.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached psy-0.0.1-py2.py3-none-any.whl.metadata (5.9 kB)
  Using cached semopy-2.3.11.tar.gz (1.6 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [40 lines of output]
      running egg_info
      creating /tmp/pip-pip-egg-info-_kvrk6nb/semopy.egg-info
      writing /tmp/pip-pip-egg-info-_kvrk6nb/semopy.egg-info/PKG-INFO
      writing dependency_links to /tmp/pip-pip-egg-info-_kvrk6nb/semopy.egg-info/dependency_links.txt
      writing requirements to /tmp/pip-pip-egg-info-_kvrk6nb/semopy.egg-info/requires.txt
      writing top-level names to /tmp/pip-pip-egg-info-_kvrk6nb/semopy.egg-info/top_level.txt
      writing manifest file '/tmp/pip-pip-egg-info-_kvrk6nb/semopy.egg-info/SOURCES.txt'
      reading manifest file '/tmp/pip-pip-egg-info-_kvrk6nb/semopy.egg-info/SOURCES.txt'
      reading manifest template 'MANIFEST.in'
      adding license file 'LICENSE'
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File

ModuleNotFoundError: No module named 'graphviz'

In [3]:
groups = {
    "Hill": ["Hill"],
}

for g in groups:
    df_map_inference_datasets = handler(str(groups[g]), 'rf', 100)
    
df_map_inference_datasets

NameError: name 'handler' is not defined

In [37]:
ccsse.df_causal_effects

,from,to,effect
0,age,hospital_number,3.627545e+06
1,age,pulse,5.837385e+01
2,age,respiratory_rate,1.702766e+01
3,rectal_temp,respiratory_rate,4.085777e+00
4,pulse,respiratory_rate,1.206101e-01
...,...,...,...
80,abdomen=distend_small,abdomen=other,-1.000000e+00
81,abdomen=firm,abdomen=other,-1.000000e+00
82,mucous_membrane=pale_cyanotic,outcome=euthanized,4.302326e-01
83,outcome=died,outcome=lived,-1.000000e+00


In [41]:
ccsse.df_error

,from,to,effect
0,hospital_number,age,0.000213
1,rectal_temp,age,0.056756
2,pulse,age,0.020809
3,respiratory_rate,age,0.002618
4,temp_of_extremities,age,0.250323
...,...,...,...
625,outcome=lived,outcome=died,NaN
626,cp_data,outcome=died,0.062883
627,outcome=lived,outcome=euthanized,NaN
628,cp_data,outcome=euthanized,0.732724


In [61]:
df_nan = ccsse.df_error[ccsse.df_error.isnull().any(axis=1)]
df_nan

,from,to,effect
16,lesion_3,age,NaN
22,mucous_membrane=pale_pink,age,NaN
25,capillary_refill_time=more_3_sec,age,NaN
30,abdomen=other,age,NaN
33,outcome=lived,age,NaN
...,...,...,...
622,outcome=lived,abdomen=other,NaN
623,cp_data,abdomen=other,NaN
625,outcome=lived,outcome=died,NaN
627,outcome=lived,outcome=euthanized,NaN


In [40]:
# Suponha que a coluna se chame 'coluna_float'
ccsse.df_causal_effects['effect_f'] = ccsse.df_causal_effects['effect'].round(3)
ccsse.df_causal_effects

,from,to,effect,effect_f
0,age,hospital_number,3.627545e+06,3627544.803
1,age,pulse,5.837385e+01,58.374
2,age,respiratory_rate,1.702766e+01,17.028
3,rectal_temp,respiratory_rate,4.085777e+00,4.086
4,pulse,respiratory_rate,1.206101e-01,0.121
...,...,...,...,...
80,abdomen=distend_small,abdomen=other,-1.000000e+00,-1.000
81,abdomen=firm,abdomen=other,-1.000000e+00,-1.000
82,mucous_membrane=pale_cyanotic,outcome=euthanized,4.302326e-01,0.430
83,outcome=died,outcome=lived,-1.000000e+00,-1.000


In [38]:
ccsse.df_causal_effects.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   from    85 non-null     object 
 1   to      85 non-null     object 
 2   effect  85 non-null     float64
dtypes: float64(1), object(2)
memory usage: 2.1+ KB


In [33]:
df_causal

,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,pain,peristalsis,abdomil_distention,...,capillary_refill_time=more_3_sec,abdomen=2,abdomen=distend_large,abdomen=distend_small,abdomen=firm,abdomen=other,outcome=died,outcome=euthanized,outcome=lived,cp_data


In [53]:
old_parents.iloc[1]['abdomen=distend_large']


0.0

In [30]:
df = dict_dfs[0]['causal_parents']
df

,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,pain,peristalsis,abdomil_distention,...,capillary_refill_time=more_3_sec,abdomen=2,abdomen=distend_large,abdomen=distend_small,abdomen=firm,abdomen=other,outcome=died,outcome=euthanized,outcome=lived,cp_data
0,0.0,528931.0,38.40,54.0,24.000000,2.000000,2.000000,1.0,3.000000,1.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,1.000000,0.0,0.0,1.0,1.0
1,0.0,528931.0,38.40,54.0,24.000000,2.000000,2.000000,1.0,3.000000,1.000000,...,0.794092,0.000000,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1.0
2,0.0,528931.0,38.40,54.0,24.000000,2.000000,2.000000,1.0,3.000000,1.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,1.000000,0.0,0.0,0.0,1.0
3,0.0,528931.0,38.40,54.0,24.000000,2.000000,2.000000,1.0,3.000000,1.000000,...,-0.111567,0.000000,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1.0
4,0.0,528931.0,38.40,54.0,24.000000,2.000000,2.000000,1.0,3.000000,1.000000,...,0.000000,0.000000,1.0,-0.532677,-0.184191,1.590119,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.0,528931.0,39.63,54.0,29.082261,2.000000,2.000000,1.0,3.000000,1.289292,...,0.111518,0.868647,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1.0
96,0.0,528931.0,38.40,54.0,24.000000,2.000000,2.000000,1.0,3.000000,1.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,1.000000,0.0,0.0,1.0,1.0
97,0.0,528931.0,38.40,98.0,29.327654,1.636786,2.055274,1.0,3.566116,2.533388,...,-0.071247,1.776908,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1.0
98,0.0,528931.0,35.57,54.0,12.494007,2.000000,2.000000,1.0,3.000000,0.521734,...,0.016368,0.915351,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1.0


In [63]:
nan_rows, nan_cols = np.where(pd.isnull(ccsse.df_error))

# Exibindo as posições onde existem NaN
print("Valores NaN encontrados nas seguintes posições:")
for row, col in zip(nan_rows, nan_cols):
    print(f"Linha: {row}, Coluna: {ccsse.df_error.columns[col]}")



Valores NaN encontrados nas seguintes posições:
Linha: 16, Coluna: effect
Linha: 22, Coluna: effect
Linha: 25, Coluna: effect
Linha: 30, Coluna: effect
Linha: 33, Coluna: effect
Linha: 50, Coluna: effect
Linha: 56, Coluna: effect
Linha: 59, Coluna: effect
Linha: 64, Coluna: effect
Linha: 67, Coluna: effect
Linha: 83, Coluna: effect
Linha: 89, Coluna: effect
Linha: 92, Coluna: effect
Linha: 97, Coluna: effect
Linha: 100, Coluna: effect
Linha: 115, Coluna: effect
Linha: 121, Coluna: effect
Linha: 124, Coluna: effect
Linha: 129, Coluna: effect
Linha: 132, Coluna: effect
Linha: 146, Coluna: effect
Linha: 152, Coluna: effect
Linha: 155, Coluna: effect
Linha: 160, Coluna: effect
Linha: 163, Coluna: effect
Linha: 176, Coluna: effect
Linha: 182, Coluna: effect
Linha: 185, Coluna: effect
Linha: 190, Coluna: effect
Linha: 193, Coluna: effect
Linha: 205, Coluna: effect
Linha: 211, Coluna: effect
Linha: 214, Coluna: effect
Linha: 219, Coluna: effect
Linha: 222, Coluna: effect
Linha: 233, Coluna: e

In [16]:
df_nan = df[df.isnull().any(axis=1)]
df_nan

,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,pain,peristalsis,abdomil_distention,...,capillary_refill_time=more_3_sec,abdomen=2,abdomen=distend_large,abdomen=distend_small,abdomen=firm,abdomen=other,outcome=died,outcome=euthanized,outcome=lived,cp_data
3,0.0,2.067234e+06,37.30,48.000000,20.000000,1.000000,2.000000,3.0,3.000000,3.000000,...,0.493447,0.000000,1.0,0.000000,0.938983,-0.114047,0.0,NaN,NaN,0.0
4,0.0,5.351580e+05,37.30,48.000000,20.000000,1.000000,2.000000,3.0,3.000000,3.000000,...,1.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.0,NaN,NaN,0.0
5,0.0,5.351580e+05,37.30,48.000000,20.000000,1.000000,2.000000,3.0,3.000000,3.000000,...,0.618724,0.000000,1.0,1.536350,0.192333,-0.882970,0.0,NaN,NaN,0.0
7,0.0,5.351580e+05,37.30,48.000000,20.000000,1.000000,2.000000,3.0,3.000000,3.000000,...,0.000000,0.000000,1.0,NaN,NaN,NaN,0.0,0.266548,1.459217,0.0
9,0.0,5.351580e+05,37.30,48.000000,20.000000,1.000000,2.000000,3.0,3.000000,3.000000,...,0.656642,0.000000,1.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0
11,0.0,5.351580e+05,37.30,48.000000,20.000000,1.000000,2.000000,3.0,3.000000,3.000000,...,0.838268,0.000000,1.0,0.000000,0.948985,-0.124048,0.0,NaN,NaN,0.0
12,0.0,5.351580e+05,37.30,48.000000,20.000000,1.000000,2.000000,3.0,3.000000,3.000000,...,1.229091,0.000000,1.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0
13,0.0,5.351580e+05,36.91,48.000000,18.463303,1.000000,2.000000,3.0,3.000000,2.983025,...,0.073552,0.887282,1.0,1.147303,0.612491,-1.117634,0.0,NaN,NaN,0.0
14,0.0,5.351580e+05,37.30,48.000000,20.000000,1.000000,2.000000,3.0,3.000000,3.000000,...,1.952292,0.000000,1.0,0.000000,0.981296,-0.156359,0.0,NaN,NaN,0.0
15,0.0,5.351580e+05,37.30,48.000000,20.000000,1.000000,2.000000,3.0,1.000000,3.637696,...,0.154708,1.962055,1.0,0.654397,0.325598,-1.412608,0.0,NaN,NaN,0.0


In [8]:
df_map_inference_datasets.iloc[17]['metrica']

'{\n    "Quantidade de inst\\u00e2ncias contrafactuais": 110,\n    "Quantidade de rela\\u00e7\\u00f5es causais na base de dados": 255,\n    "Quantidade de atributos modificados": 0,\n    "Quantidade de inst\\u00e2ncias contrafactuais causais": 0,\n    "Quantidade de rela\\u00e7\\u00f5es causais analisadas": 0,\n    "Quantidade de rela\\u00e7\\u00f5es causais satisfeitas": 0,\n    "Quantidade de inst\\u00e2ncias contrafactuais com um \\u00fanico atributo modificado": 0,\n    "Tempo de execu\\u00e7\\u00e3o": 106.5670256614685\n}'

In [2]:
dfm_use = pd.read_parquet(f"s3://omar-testes-gerais/artigos/artifacts/dfm_use.parquet")
dfm_use

,name,path,classe,metrica,metrica_v2
0,Adult,adult/adult_processada.csv,Above/Below 50K,memoria 18gb,None
1,Australian,Australian/Credit_Card_Applications.csv,Class,"{""Quantidade de inst\u00e2ncias contrafactuais...",None
2,Banknote,Banknote/BankNote_Authentication.csv,class,"{""Quantidade de inst\u00e2ncias contrafactuais...",None
3,Biodeg,Biodeg/qsar-biodeg.csv,Class,"{\n ""Quantidade de instâncias contrafactuai...",zero metricas
4,Breast Cancer Coimbra,Breast Cancer Coimbra/breast_coimbra.csv,Classification,"{""Quantidade de inst\u00e2ncias contrafactuais...",None
5,Breast Cancer Wisconsin,Breast Cancer Wisconsin/breast-cancer.csv,diagnosis,"{""Quantidade de inst\u00e2ncias contrafactuais...",None
6,Churn,Churn/WA_Fn-UseC_-Telco-Customer-Churn_process...,Churn,"Input contains NaN, infinity or a value too la...",None
7,Compas,Compas/compas-scores-two-years.csv,two_year_recid,base de dados nao processada,None
8,Credit default,Credit default/UCI_Credit_Card.csv,default.payment.next.month,memoria,None
9,Diabetes,Diabetes/diabetes.csv,Outcome,"{""Quantidade de inst\u00e2ncias contrafactuais...",None


In [5]:
path = 's3://omar-testes-gerais/artigos/artifacts/datasets'
df_use_index = 16
# dfx = pd.read_csv(f"{path}/{dfm_use.iloc[df_use_index]['path']}", nrows=None)
dfx = pd.read_csv(f"{path}/{dfm_use.iloc[df_use_index]['path']}")

# dfx = pd.read_csv(f"{path}/{dfz.iloc[df_use_index]['path']}")
dfx

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,pain,peristalsis,...,capillary_refill_time=more_3_sec,abdomen=2,abdomen=distend_large,abdomen=distend_small,abdomen=firm,abdomen=other,outcome=died,outcome=euthanized,outcome=lived,cp_data
0,0,0,530101,38.5,66,28,1,1,5,4,...,1,0,1,0,0,0,1,0,0,0
1,1,0,534817,39.2,88,20,1,1,3,4,...,0,0,0,0,0,1,0,1,0,0
2,0,0,530334,38.3,40,24,2,2,3,3,...,0,1,0,0,0,0,0,0,1,1
3,1,1,5290409,39.1,164,84,0,2,2,4,...,1,0,1,0,0,0,1,0,0,1
4,0,0,530255,37.3,104,35,1,1,2,2,...,1,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,0,0,533902,38.5,40,16,2,2,2,1,...,0,0,0,0,0,1,0,0,1,0
235,0,0,527702,37.2,72,24,1,3,4,3,...,1,0,0,1,0,0,0,1,0,1
236,1,0,529386,37.5,72,30,0,1,4,4,...,0,0,1,0,0,0,1,0,0,0
237,1,0,530612,36.5,100,24,1,1,3,3,...,0,0,0,1,0,0,0,0,1,1


In [6]:
dfx[dfx.isna()]

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,pain,peristalsis,...,capillary_refill_time=more_3_sec,abdomen=2,abdomen=distend_large,abdomen=distend_small,abdomen=firm,abdomen=other,outcome=died,outcome=euthanized,outcome=lived,cp_data
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
236,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
237,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
